In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import dlib
import pickle 
import matplotlib.pyplot as plt 
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
TikaiNetV1 = tf.keras.models.load_model('TikaiNetV1')
#scaler = pickle.load(open('scaler.pkl', 'rb'))
face_detector = dlib.get_frontal_face_detector()

In [ ]:

datatest = ImageDataGenerator(
    preprocessing_function = preprocess_input
)

X_test = datatest.flow_from_directory(
    "dataset_2/test_set",
    target_size=(224,224),
    color_mode='grayscale',
    batch_size=32,
    class_mode="sparse",
    shuffle=True,
    seed=213,
)

i=0
for X,y in X_test:
    for image in X:
        i+=1
        plt.figure()
        plt.imshow(image)
        image = image.reshape(1,224,224,1)
        A = TikaiNetV1.predict(image)
        Z = A[0]
        if Z[0]>Z[1]:
            print("Il a")
        else :
            print("Il a pas")
        if i == 10:
            break
    break

In [ ]:
def draw_mask(rgb,x,y,w,h,v=20,color=(255,0,0),thickness=2,mask=True,prob=0):
    font = cv2.FONT_HERSHEY_SIMPLEX

    if mask :
        cv2.putText(rgb,f"Mask {round(prob* 100,2)} %",(x,y-fit),font,1,thickness=3,color=(0,255,0))
    else : 
        cv2.putText(rgb,f"No Mask {round(prob* 100,2)} %",(x,y-fit),font,1,thickness=3,color=(0,0,255))

    cv2.line(rgb,(x,y),(x+v,y),color,thickness)
    cv2.line(rgb,(x,y),(x+v,y),color,thickness)

    cv2.line(rgb,(x+w,y),(x+w-v,y),color,thickness)
    cv2.line(rgb,(x+w,y),(x+w,y+v),color,thickness)

    cv2.line(rgb,(x,y+h),(x,y+h-v),color,thickness) 
    cv2.line(rgb,(x,y+h),(x+v,y+h),color,thickness)

    cv2.line(rgb,(x+w,y+h),(x+w,y+h-v),color,thickness)
    cv2.line(rgb,(x+w,y+h),(x+w-v,y+h),color,thickness)


In [ ]:
cap = cv2.VideoCapture(0)
fit = 20

while(True):
    ret, frame = cap.read()
    nb_faces = face_detector(frame)
    if len(nb_faces)>0:
        for counter,face in enumerate(nb_faces):
            try : 
                x1,y1 = face.left(),face.top()
                x2,y2 = face.right(),face.bottom()

                x = x1-fit
                y = y1-fit 
                w = x2+fit 
                h = y2+fit 
                
            
                img_crop = frame[y:h,x:w]
                img_crop = cv2.cvtColor(img_crop, cv2.COLOR_BGR2GRAY)
                img_crop = cv2.resize(img_crop,(224,224))
                img_crop = img_to_array(img_crop)
                X = preprocess_input(img_crop)

                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.rectangle(frame,(x1,y1),(x2,y2),(220,255,220),1)

                print(X.max())
                X = X.reshape(-1,224,224,1)
                Z = TikaiNetV1.predict(X)
                print(Z)
                A = Z[0]

                if A[0]>A[1]:
                    draw_mask(frame,x1,y1,x2-x1,y2-y1,10,(0,255,0),3,mask=True,prob=A[0])
                else:
                    draw_mask(frame,x1,y1,x2-x1,y2-y1,10,(0,0,255),3,mask=False,prob=A[1])


            except Exception as e :
                print("Erreur")
    else :  
        font = cv2.FONT_HERSHEY_SIMPLEX
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        X = cv2.resize(gray,(224,224))
        X = preprocess_input(X)
        print(X.max())
        X = X.reshape(-1,224,224,1)
        Z = TikaiNetV1.predict(X)
        print(Z)
        Y = Z[0]
        if Y[0]>Y[1]:
            cv2.putText(frame,f"Mask {round(Y[0]* 100,2)} %",(10,450),font,1,thickness=3,color=(0,255,0))
        else:
            cv2.putText(frame,f"No Mask {round(Y[1]*100,2)} %",(10,450),font,1,thickness=3,color=(0,0,255))


    cv2.imshow('frame', frame)
    cv2.waitKey(50)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
